In [ ]:
%matplotlib notebook
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from matplotlib import animation

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

# Diseño de sistemas y filtros FIR

En la lección anterior definimos un sistema FIR como

$$
y[n] = (h * x)[n]
$$

donde $h$ es un vector de largo $L+1$ que tiene los coeficientes del sistema

El filtro FIR observa $L+1$ instantes de la entrada para calcular la salida

En esta lección veremos

- La respuesta al impulso y respuesta en frecuencia de un sistema
- La definición de filtro y los tipos básicos de filtros
- Como diseñar un filtro FIR, es decir decidir los valores del vector $h$

## Respuesta al impulso de un sistema


Sea el impulso unitario o delta de Kronecker

$$
\delta[n-m] = \begin{cases} 1 & n=m \\ 0 & n \neq m \end{cases}
$$

La **respuesta al impulso de un sistema discreto** es la salida obtenida cuando la entrada es un impulso unitario


Para un sistema FIR arbitrario tenemos

$$
y[n]|_{x=\delta} = (h * \delta)[n] = \sum_{j=0}^L h[j] \delta[n-j] = \begin{cases} h[n] & n \in [0, L] \\ 0 & \text{en otro caso} \end{cases} \\
$$

es decir que la respuesta al impulso:

-  tiene **una duración finita y luego decae a zero**
-  recupera los coeficientes $h[j]$ del sistema

En un sistema causal se tiene que $h[n] = 0 \quad \forall n < 0$

Llamamos **soporte** del sistema a todos aquellos valores de $n$ tal que  $h[n] \neq 0$


### Ejemplo

Para el sistema reverberante

$$
y[n] = x[n] + A x[n-m]
$$

la respuesta al impulso es

$$
y[n] = \delta[n] + A \delta[n-m] = \begin{cases} 1 & n=0\\ A& n=m \\ 0 & \text{en otro caso} \end{cases}
$$

La respuesta al impulse me permite a recuperar los coeficientes del sistema en caso de que no los conociera

## Respuesta en frecuencia de un sistema

Sea un sistema lineal cuyos coeficientes no cambian en el tiempo, como el sistema FIR que hemos estado estudiando

Por propiedad de la transformada de Fourier sabemos que 

$$
\begin{align}
\text{DFT}_N [y[n]] & = \text{DFT}_N [(h * x)[n]] \nonumber \\
\text{DFT}_N [y[n]] & = \text{DFT}_N [h[n]]  \cdot \text{DFT}_N [x[n]] \nonumber \\
Y[k] &= H[k] \cdot X[k] , 
\end{align}
$$

donde llamamos a $H[k]$ la **respuesta en frecuencia del sistema** 

La respuesta en frecuencia es **la transformada de Fourier de la respuesta al impulso**


### Ejemplo: Respuesta en frecuencia del sistema promediador

Podemos calcular la respuesta en frecuencia de un filtro a partir de su respuesta al impulso $h$ usando la función

```python
scipy.signal.freqz(b, # Coeficientes en el numerador h
                   a=1, # Coeficientes en el denominador de h
                   fs=6.28318 # Frecuencia de muestreo
                   ...
                  )
```

Para el caso de un filtro FIR solo existen coeficientes en el numerador por lo que no utilizamos el argumento $a$

La función retorna 

```python
    freq, H = scipy.signal.freqz(b=h)
```

un arreglo de frecuencias y la respuesta en frecuencia (compleja)

Por ejemplo el sistema promediador que vimos anteriormente

$$
h[i] = \begin{cases} 1/L & i < L \\ 0 & i > L \end{cases}
$$

el valor absoluto de su respuesta en frecuencia es

In [ ]:
from bokeh.palettes import Dark2_5 as palette

p = [Figure(plot_width=600, plot_height=230, toolbar_location="below") for k in range(2)]

for L, color in zip([10, 20, 50], palette):

    h = np.zeros(shape=(100,))
    h[:L] = 1/L
    freq, H = scipy.signal.freqz(b=h, fs=1)
    p[0].line(range(100), h, color=color, line_width=2)
    p[1].line(freq, np.absolute(H), 
              color=color, line_width=2, legend_label=f"L={L}")

show(column(p))

Mientras más ancho es el sistema en el dominio del tiempo ($L$ grande), más concentrada se vuelve su respuesta en frecuencia

Si multiplicamos $H$ con el espectro de una señal, lo que estamos haciendo es atenuar las frecuencias altas

Formalicemos este concepto a continuación

## Filtros digitales

Un **filtro** es un sistema cuyo objetivo es reducir o resaltar un aspecto específico de una señal

Por ejemplo

- Disminuir el nivel de ruido
- Separar dos o más señales que están mezcladas
- Ecualizar la señal
- Restaurar la señal (eliminar desenfoque o artefactos de grabación)

Llamamos **filtro digital** a los filtros aplicados a señales digitales

Hablamos de **señal filtrada** para referirnos a la salida del filtro

En esta unidad nos enfocaremos en filtros cuyos coeficientes son fijos y no se modifican en el tiempo. En la próxima unidad veremos filtros que se adaptan continuamente a los cambios de la entrada

## Tipos básicos de filtro 

Como vimos el filtro lineal e invariante en el tiempo puede estudiarse en frecuencia usando 

$$
Y[k] = H[k] X[k] ,
$$

donde $H[k]$ es la DFT del filtro (respuesta en frecuencia)

El filtro actua como una **máscara multiplicativa** que modifica el espectro de la señal entrada

Esto significa que solo puede acentuar, atenuar o remover ciertas frecuencias pero **nunca crear nuevas frecuencias**

Consideremos los siguienes filtros o máscaras ideales



<img src="../images/ideal_filters.gif">

- Filtro pasa bajo: Anula las frecuencias altas. Sirve para suavizar
- Filtro pasa alto: Anula las frecuencias bajas. Sirve para detectar cambios
- Filtro pasa banda: Anula todo excepto una banda continua de frecuencias
- Filtro rechaza banda: Anula sólo una banda continua de frecuencias

Las llamamos "ideales" por que en general los cortes de los filtros no pueden ser tan abruptos como se muestra en la figura

A continuación veremos un método para diseñar filtros FIR partiendo desde el dominio de la frecuencia

## Diseño de un filtro FIR: Método de la ventana

Diseñar un filtro consisten en definir 

- L: El largo de la respuesta al impulso
- h: Los valores de la respuesta al impulso

El siguiente algoritmo de diseño de filtro se llama el "método de la ventana" y parte de la base de una respuesta en frecuencia ideal

1. Especificar una **respuesta en frecuencia** ideal $H_d[k]$ dependiendo de los requerimientos
1. Usar la transformada de Fourier inversa para obtener la **respuesta al impulso ideal** $h_d[n]$
1. Truncar la respuesta al impulso ideal usando **una ventana** tal que $h[n] = h_d[n] w[n]$

Finalmente $h[n]$ nos da los coeficientes del filtro FIR y $w[n]$ nos da el largo del filtro

La ventana $w[n]$ puede ser cualquiera de las funciones vistas en la unidad anterior, por ejemplo una ventana rectangular

$$
w[n] = \begin{cases} 1 & n \leq L \\ 0 & n > L \end{cases}
$$

o la ventana de Hann

$$
w[n] = 0.5 - 0.5 \cos \left( \frac{2\pi n}{L-1} \right)
$$

### Diseño de un filtro pasa bajo (LPF)

Un filtro pasa bajo es aquel que sólo deja pasar las **bajas** frecuencias

Sus usos son:

- Recuperar una tendencia o comportamiento lento en la señal
- Suavizar la señal y disminuir la influencia del ruido aditivo

Diseñemos un filtro que elimine todas las frecuencias mayores a $f_c$ [Hz] de una señal $x[n]$ muestreada con frecuencia $F_s$


#### Respuesta en frecuencia ideal

Propongamos la siguiente respuesta en frecuencia que solo deja pasar las frecuencias menores a $f_c$, es decir que sólo es distinta de cero en el rango $[-f_c, f_c]$

$$
\begin{align}
H_d(\omega) &= \begin{cases} 1 & |f| < f_c\\ 0 & |f| > f_c \end{cases} \nonumber \\
&= \text{rect}(f/f_c) \nonumber 
\end{align}
$$



#### Respuesta al impulso ideal

Obtenemos la transformada de Fourier inversa de la respuesta en frecuencia

$$
\begin{align}
h_d(t) &=  \int_{-f_c}^{f_c} e^{j 2 \pi f t} df \nonumber \\
& = \frac{2j  f_c}{2 j \pi f_c t} \sin(2 \pi f_c t) = 2  f_c \text{sinc}(2 \pi f_c t) \nonumber 
\end{align}
$$

donde la versión en tiempo discreto sería

$$
h_d[n] = 2 f_c\text{sinc}(2 \pi f_c n/ F_s)/F_s 
$$

Notemos que es una función infinitamente larga


In [ ]:
fc = 0.1 # Frecuencia de corte
Fs = 1 # Frecuencia de muestreo
n = np.arange(-50, 50, step=1/Fs);
f = sfft.fftshift(sfft.fftfreq(n=len(n), d=1/Fs))

# Diseño de la respuesta en frecuencia ideal
kc = int(len(n)*fc)
Hd = np.zeros_like(n, dtype=np.float); 
Hd[:kc] = 1
Hd[len(Hd)-kc+1:] = 1
# Cálculo de la respuesta al impulso ideal
#hd = np.real(sfft.ifftshift(sfft.ifft(Hd)))
hd = 2*fc*np.sinc(2*fc*n/Fs)/Fs # Se omite Pi por que está incluido en np.sinc

p = [Figure(plot_width=300, plot_height=230, toolbar_location="below") for k in range(2)]
p[0].line(f, sfft.fftshift(Hd), line_width=2)
p[0].title.text = 'Respuesta en frecuencia ideal'
p[1].line(n, hd, line_width=2)
p[1].title.text = 'Respuesta al impulso ideal'
show(row(p))

#### Truncar la respuesta al impulso ideal

Para obtener una respuesta al impulso finita multiplicamos por una ventana finita de largo $L+1$

$$
h[n] = 2  f_c \text{sinc}(2 \pi f_c n /F_s) \cdot \text{rect}(n/(L+1))
$$

In [ ]:
# Cálculo de la respuesta al impulso truncada

def truncar(hd, L=100):
    w = np.zeros_like(hd); 
    w[len(w)//2-L//2:len(w)//2+L//2+1] = 1.
    return w*hd
# Cálculo de la respuesta en frecuencia truncada
h = truncar(hd)
H = sfft.fft(h); 

p = [Figure(plot_width=300, plot_height=230, toolbar_location="below") for k in range(2)]
p[1].line(f, sfft.fftshift(np.absolute(H)), line_width=2)
p[1].title.text = 'Respuesta en frecuencia truncada'
p[0].line(n, h, line_width=2)
p[0].title.text = 'Respuesta al impulso truncada'
show(row(p))

Comparemos la respuesta en frecuencia ideal con la que en realidad aplicamos a la señal

In [ ]:
p = Figure(plot_width=600, plot_height=250, toolbar_location="below")
p.line(f, sfft.fftshift(Hd), color=palette[0],
       line_width=2, legend_label="Ideal")
for i, L in enumerate([20, 40]):
    H = sfft.fft(truncar(hd, L))
    p.line(f, sfft.fftshift(np.absolute(H)), color=palette[i+1],
           line_width=2, legend_label=f"Truncada L={L}")
p.xaxis.axis_label = 'Frecuencia [Hz]'
show(p)

La respuesta en frecuencia "ideal" $H_d[k]$ es plana y tiene discontinuidades 

La respuesta en frecuencia "real" $H[k]$ busca aproximar a $H_d[k]$. Pero observando $H[k]$ notamos que

- No tiene zonas perfectamente planas (ni es perfectamente zero)
- No tiene discontinuidades fuertes como el caso ideal

Esto se debe al recorte que hacemos con la ventana

<img src="../images/system-real-filter.png" width="500">

En general mientras más grande es $L$ más fiel será la respuesta en frecuencia

También se puede usar una ventana más suave (por jemeplo Hann) para disminuir los *ripples* al costo de hacer más lenta las transiciones



### Diseño de un filtro pasa alto (HPF)

Un filtro pasa alto es aquel que sólo deja pasar las **altas** frecuencias

Sus usos son:
- Identificar cambios/detalles, es decir comportamientos rápidos en una señal
- Eliminar tendencias

Podemos diseñar un filtro pasa alto a partir de un filtro pasa bajo usando el truco de **inversión espectral**, como muestra la siguiente figura

<img src="../images/system-hpf.png" width="400">

Basicamente estamos **restándole un filtro pasa bajo a un impulso unitario**, como muestra el siguiente código. Note la respuesta en frecuencia del filtro pasa alto resultante


In [ ]:
Fs = 10 # Frecuencia de muestreo
fc = 2 # Frecuencia de corte
L = 100+1 # Largo del filtro

# Filtro pasa bajo
t = np.arange(-L//2, L//2, step=1)/Fs
h = 2*fc*np.sinc(2*fc*t)/Fs 
# Filtro pasa alto
impulso = np.zeros_like(h)
impulso[L//2+1] = 1
h = impulso - h
freq, H = scipy.signal.freqz(h, fs=Fs)

p = [Figure(plot_width=300, plot_height=230, toolbar_location="below") for k in range(2)]
p[0].line(t, h, line_width=2)
p[1].line(freq, np.absolute(H), line_width=2)    
p[1].title.text = 'Respuesta en frecuencia'
p[0].title.text = 'Respuesta al impulso'
show(row(p))

### Diseño de un filtro pasa banda (BPF) y rechaza banda (BRF)

Como sus nombres lo indican estos filtros 
- BPF: Dejan pasar sólo una cierta banda de frecuencia 
- BRF: Dejan pasar todas las frecuencias excepto una banda determinada 

La banda de frecuencia está definida por sus frecuencias de corte mínima y máxima $f_{c1} < f_{c2}$

Un filtro pasa banda se puede construir convolucionando la señal con un filtro pasa bajo con frecuencia de corte $f_{c2}$ y luego convolucionando el resultado con un filtro pasa alto con frecuencia de corte $f_{c1}$ como muestra  la siguinte figura


<img src="../images/system-bpf.png" width="400">

In [ ]:
Fs = 10 # Frecuencia de muestreo
fc1, fc2 = 2, 3 # Frecuencias de cortes
L = 100+1 # Largo del filtro

# Filtro pasa bajo con frecuencia fc2
t = np.arange(-L//2, L//2, step=1)/Fs
h1 = 2*fc2*np.sinc(2*fc2*t)/Fs 
# Filtro pasa alto con frecuencia fc1
h2 = 2*fc1*np.sinc(2*fc1*t)/Fs 
impulso = np.zeros_like(h2)
impulso[L//2+1] = 1
h2 = impulso - h2
# FIltro pasa banda como convolución
h = scipy.signal.convolve(h1, h2, mode='same')
freq, H = scipy.signal.freqz(h, fs=Fs)

p = [Figure(plot_width=300, plot_height=230, toolbar_location="below") for k in range(2)]
p[0].line(t, h, line_width=2)
p[1].line(freq, np.absolute(H), line_width=2)    
p[1].title.text = 'Respuesta en frecuencia'
p[0].title.text = 'Respuesta al impulso'
show(row(p))

Un filtro rechaza banda en cambio se puede construir sumando las respuestas al impulso de un filtro pasa bajo con frecuencia de corte $f_{c2}$ con una filtro pasa alto de frecuencia de corte $f_{c1}$ como muestra la siguiente figura

<img src="../images/system-rbf.png" width="400">

In [ ]:
Fs = 10 # Frecuencia de muestreo
fc1, fc2 = 2, 3 # Frecuencias de cortes
L = 100+1 # Largo del filtro

# Filtro pasa bajo con frecuencia fc1
t = np.arange(-L//2, L//2, step=1)/Fs
h1 = 2*fc1*np.sinc(2*fc1*t)/Fs 
# Filtro pasa alto con frecuencia fc2
h2 = 2*fc2*np.sinc(2*fc2*t)/Fs 
impulso = np.zeros_like(h2)
impulso[L//2+1] = 1
h2 = impulso - h2
# FIltro rechaza banda como suma
h = h1 + h2
freq, H = scipy.signal.freqz(h, fs=Fs)

p = [Figure(plot_width=300, plot_height=230, toolbar_location="below") for k in range(2)]
p[0].line(t, h, line_width=2)
p[1].line(freq, np.absolute(H), line_width=2)    
p[1].title.text = 'Respuesta en frecuencia'
p[0].title.text = 'Respuesta al impulso'
show(row(p))

## Diseño usando scipy

Podemos diseñar un filtro usando la técnica de enventando con la función de scipy

```python
scipy.signal.firwin(numtaps, # (entero) Largo del filtro
                    cutoff, # Frecuencia de corte: Una para pasa bajo/alto o dos para pasa/rechaza banda
                    window='hamming', # Tipo de ventana
                    pass_zero=True, # pasa bajo (True) o pasa alto (False)
                    fs=None # Frecuencia de muestreo
                    ...
                    )
```
Esta función retorna un arreglo con $h$, la respuesta al impulso del filtro FIR

En la siguiente actividad aprenderemos a usar esta función

## Resumen

El diseño del filtro está dado entonces por su

- **Función:** Definida por la respuesta en frecuencia ideal 
- **Fidelidad:** El error tolerable entre la respuesta en frecuencia ideal y la real

El tipo de filtro y sus frecuencias de corte definen su función. Esto es un requisito del problema que buscamos resolver.

El parámetro $L$ nos da un trade-off para la fidelidad. Si agrandamos $L$ tendremos mayor fidelidad pero más costo computacional




